In [ ]:
# -*- coding: utf-8 -*-
# @Author : Xingmin Liu
# @Time : 2023.11.16

In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset, random_split
from torchvision import transforms
import torch

class RealORFakeDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

        # 对非数值型特征进行编码
        for column in ['is_noisy', 'has_repeating_patterns', 'is_symmetric']:
            self.data[column] = self.data[column].replace({'True': 1, 'False': 0})

    def __getitem__(self, index):
        # 加载并预处理图像
        image = Image.open(self.data.iloc[index]['image_path']).convert('RGB')
        image = self.transform(image)

        # 获取其他特征
        other_features = self.data.iloc[index].drop(['image_path', 'label']).values.astype(float)
        other_features = torch.tensor(other_features, dtype=torch.float)

        # 获取标签
        label = self.data.iloc[index]['label']
        label = torch.tensor(label, dtype=torch.long)

        return image, other_features, label

    def __len__(self):
        return len(self.data)

In [2]:
from torch.utils.data import DataLoader
import pandas as pd

# 定义图像预处理
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomVerticalFlip(),  # 随机垂直翻转
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomRotation(40),  # 在[-30, 30]范围内随机旋转
    transforms.RandomAffine(degrees=0, shear=10, scale=(0.8,1.2)),  # 随机仿射变换
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # 色彩抖动
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(),  # 随机擦除
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 创建数据集实例
train_dataset = RealORFakeDataset('./_train_final_s.csv', transform=train_transform)
valid_dataset = RealORFakeDataset('./_valid_final_s.csv', transform=valid_transform)

# 创建 DataLoader 实例
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class MyModel(nn.Module):
    def __init__(self, num_other_features):
        super(MyModel, self).__init__()

        # 图像输入分支 (ResNet50)
        self.resnet = models.resnet50(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # Remove the final fc layer.

        # 非图像输入分支
        self.other_features_layer = nn.Sequential(
            nn.Linear(num_other_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
        )

        # 输出层
        self.output_layer = nn.Sequential(
            nn.Linear(num_features + 128, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, image_inputs, other_inputs):
        image_features = self.resnet(image_inputs)
        other_features = self.other_features_layer(other_inputs)
        concatenated = torch.cat([image_features, other_features], dim=1)
        return self.output_layer(concatenated)

In [4]:
import torch
from torch import optim

# 假设 MyModel 已经被定义
num_other_features = 12
model = MyModel(num_other_features=num_other_features)

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)


/environment/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/environment/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import torch
from sklearn.metrics import f1_score

from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def train(model, train_loader, val_loader, optimizer, num_epochs):
    # 切换模型到训练模式
    model.train()

    # 用于保存每个批次的 loss 和 F1 分数
    train_losses = []
    train_f1s = []
    val_losses = []
    val_f1s = []

    for epoch in range(num_epochs):
        batch_losses = []
        epoch_labels = []
        epoch_outputs = []

        # 添加 tqdm 进度条
        for images, other_features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Training)"):
            # 将数据移动到设备上
            images = images.to(device)
            other_features = other_features.to(device)
            labels = labels.to(device)

            # 清空优化器的梯度
            optimizer.zero_grad()

            # 前向传播
            outputs = model(images, other_features)
            loss = torch.nn.BCELoss()(outputs.squeeze(), labels.float())

            # 反向传播和优化
            loss.backward()
            optimizer.step()

            # 保存每个批次的 loss
            batch_losses.append(loss.item())

            # 保存输出和标签以计算 F1 分数
            epoch_labels.extend(labels.detach().cpu().numpy())
            epoch_outputs.extend((outputs.detach().cpu().numpy() > 0.5).astype(int))  # 设置阈值为 0.5

        # 保存本 epoch 的所有批次的 loss
        train_losses.append(batch_losses)

        # 计算本 epoch 的 F1 分数，然后保存
        epoch_f1 = f1_score(epoch_labels, epoch_outputs, average='macro')
        train_f1s.append(epoch_f1)

        # 评估验证集
        model.eval()  # 切换模型为评估模式
        with torch.no_grad():
            val_epoch_losses = []
            val_epoch_labels = []
            val_epoch_outputs = []

            for val_images, val_other_features, val_labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)"):
                # 将数据移动到设备上
                val_images = val_images.to(device)
                val_other_features = val_other_features.to(device)
                val_labels = val_labels.to(device)

                # 前向传播
                val_outputs = model(val_images, val_other_features)
                val_loss = torch.nn.BCELoss()(val_outputs.squeeze(), val_labels.float())

                # 保存 loss
                val_epoch_losses.append(val_loss.item())

                # 保存输出和标签以计算 F1 分数
                val_epoch_labels.extend(val_labels.detach().cpu().numpy())
                val_epoch_outputs.extend((val_outputs.detach().cpu().numpy() > 0.5).astype(int))  # 设置阈值为 0.5

            # 计算本 epoch 的平均 loss 和 F1 分数，然后保存
            val_epoch_loss = sum(val_epoch_losses) / len(val_epoch_losses)
            val_losses.append(val_epoch_loss)
            val_epoch_f1 = f1_score(val_epoch_labels, val_epoch_outputs, average='macro')
            val_f1s.append(val_epoch_f1)

        model.train()  # 切换模型回训练模式

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {sum(batch_losses)/len(batch_losses)}, Train F1 Score: {epoch_f1}, Val Loss: {val_epoch_loss}, Val F1 Score: {val_epoch_f1}')

    return train_losses, train_f1s, val_losses, val_f1s

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for val_images, val_other_features, val_labels in valid_loader:
    print(val_other_features.shape)
    # 将数据移动到设备上
    val_images = val_images.to(device)
    val_other_features = val_other_features.to(device)
    val_labels = val_labels.to(device)
    break

torch.Size([256, 12])


In [7]:
# 检查是否有可用的 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 将模型移到设备上
model = model.to(device)

# 开始训练
# train_losses, train_f1s, val_losses, val_f1s = train(model, train_loader, valid_loader, optimizer, num_epochs=10)

Epoch 1/10 (Validation): 100%|██████████| 13/13 [00:12<00:00,  1.03it/s]


Epoch 1/10, Train Loss: 0.1216692403703928, Train F1 Score: 0.9509373550639639, Val Loss: 0.0588879083068325, Val F1 Score: 0.9809340573312505


Epoch 2/10 (Validation): 100%|██████████| 13/13 [00:12<00:00,  1.08it/s]


Epoch 2/10, Train Loss: 0.055252220816910266, Train F1 Score: 0.9815624635390514, Val Loss: 0.030736291136306066, Val F1 Score: 0.9890617212885566


Epoch 3/10 (Validation): 100%|██████████| 13/13 [00:11<00:00,  1.12it/s]


...

Epoch 8/10, Train Loss: 0.03193630635738373, Train F1 Score: 0.988828105293716, Val Loss: 0.02348239958071365, Val F1 Score: 0.9909372442249591


Epoch 9/10 (Validation): 100%|██████████| 13/13 [00:12<00:00,  1.02it/s]


Epoch 9/10, Train Loss: 0.028129221154376865, Train F1 Score: 0.990078117672438, Val Loss: 0.016159846178757455, Val F1 Score: 0.9943749978027335


Epoch 10/10 (Validation): 100%|██████████| 13/13 [00:12<00:00,  1.06it/s]

Epoch 10/10, Train Loss: 0.026241997880861165, Train F1 Score: 0.9908593736052511, Val Loss: 0.021637405738986742, Val F1 Score: 0.991561984984435


In [ ]:
import matplotlib.pyplot as plt

# 假设 train_losses, train_f1s, val_losses, val_f1s 是四个列表，包含了训练和验证过程的损失和 F1 分数

# 创建一个新的 figure
plt.figure(figsize=(12, 8))

# 绘制训练损失
plt.subplot(2, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# 绘制训练 F1 分数
plt.subplot(2, 2, 2)
plt.plot(train_f1s, label='Training F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Training F1 Score')
plt.legend()

# 绘制验证损失
plt.subplot(2, 2, 3)
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Validation Loss')
plt.legend()

# 绘制验证 F1 分数
plt.subplot(2, 2, 4)
plt.plot(val_f1s, label='Validation F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Validation F1 Score')
plt.legend()

# 显示 figure
plt.tight_layout()
plt.show()

In [9]:
# 保存模型
model_path = "MutiResNet.pth"
torch.save(model, model_path)

In [6]:
from PIL import Image
from torchvision import transforms
from utils import noise
from utils import repeat
from utils import symmetric_sift
from utils import naturalF

# is_noisy
# noise_ratio
# noise_level
# has_repeating_patterns
# is_symmetric
# good_matches_count
# brightness
# contrast
# saturation
# h_entropy
# s_entropy
# v_entropy
# label
# TRUE,0.202346802,3,TRUE,FALSE,19,114.3035736,56.91729966,102.1498413,5.090866089,6.892580509,7.659484863,0
# 定义一个函数来对单张图片进行预测
from PIL import Image
def predict_image(model, image_path):
    model.eval()  # 切换模型为评估模式
    # 加载图片
    image = Image.open(image_path)
    if image.format == 'PNG':
        # 将 RGBA 转换为 RGB，忽略 alpha 通道
        image = image.convert('RGB')
    _, _, _, noise_ratio, is_noisy, noise_level = noise.noise_detection_17_40(image_path)
    is_noisy = int(is_noisy)
    noise_level = int(noise_level)
    _, has_repeating_patterns, _ = repeat.detect_repeating_patterns_21_30(image_path)
    has_repeating_patterns = int(has_repeating_patterns)
    is_symmetric, good_matches_count = symmetric_sift.is_symmetric_sift_16_55(image_path)
    is_symmetric = int(is_symmetric)
    check = naturalF.is_natural_color_histogram_advanced_14_00(image_path)
    brightness, contrast, saturation, h_entropy, s_entropy, v_entropy = check.values()
    other_input = [[noise_ratio, is_noisy, noise_level, has_repeating_patterns, is_symmetric, good_matches_count, brightness, contrast, saturation, h_entropy, s_entropy, v_entropy]]
    # print(other_input)
    other_input_tensor = torch.Tensor(other_input)

    # 定义转换
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 使用与训练模型时相同的归一化参数
    ])

    # 对图片进行转换
    image = transform(image)

    # 添加一个维度来表示批量大小（因为 PyTorch 总是期望有一个批量维度）
    image = image.unsqueeze(0)

    # 将图片移动到设备上
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image = image.to(device)
    other_input_tensor = other_input_tensor.to(device)

    # 使用模型进行预测
    output = model(image, other_input_tensor)
    output = output.item()
    if output > 0.5:
        output_class = 1
    else:
        output_class = 0

    # 返回预测结果
    return output, output_class

In [8]:
import os
import csv
import time
import torch
from tqdm import tqdm

def predict_images_in_folder(model, folder_path, output_csv_path):
    # 获取文件夹中的所有文件
    image_files = os.listdir(folder_path)

    # 创建 CSV 文件，准备写入预测结果
    with open(output_csv_path, 'w', newline='') as csvfile:
        fieldnames = ['image_path', 'prediction', 'prediction_class']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        # 记录开始时间
        start_time = time.time()

        # 对每个文件进行预测
        for image_file in tqdm(image_files, desc="Predicting images"):
            image_path = os.path.join(folder_path, image_file)
            prediction, prediction_class = predict_image(model, image_path)

            # 写入预测结果
            writer.writerow({'image_path': image_path, 'prediction': prediction, 'prediction_class': prediction_class})

        # 计算并输出预测时间
        elapsed_time = time.time() - start_time
        print(f'Total prediction time: {elapsed_time} seconds')

# 使用方法
model = torch.load('./MutiResNet_1116.pth')
folder_path = './input/newest_test/'  # 图片文件夹的路径
output_csv_path = './output/predictions.csv'  # 输出 CSV 的路径
predict_images_in_folder(model, folder_path, output_csv_path)

Predicting images:   2%|▏         | 90/4000 [03:01<2:05:00,  1.92s/it]

Not enough descriptors found in one or both of the images. Skipping ./input/newest_test/test_1079.jpg


Predicting images:  13%|█▎        | 509/4000 [17:30<2:03:55,  2.13s/it]

No descriptors found in one or both of the images. Skipping ./input/newest_test/test_1456.jpg


Predicting images:  16%|█▌        | 635/4000 [21:46<1:44:10,  1.86s/it]

...

No descriptors found in one or both of the images. Skipping ./input/newest_test/test_801.jpg


Predicting images:  99%|█████████▉| 3977/4000 [2:08:21<00:42,  1.84s/it]

Not enough descriptors found in one or both of the images. Skipping ./input/newest_test/test_978.jpg


Predicting images: 100%|██████████| 4000/4000 [2:09:06<00:00,  1.94s/it]

Total prediction time: 7746.987056493759 seconds


In [24]:
predict_image(model, image_path='./data/dataset/test/real/test_real_4.jpg')

0.9982004165649414